In [1]:
import chess
import pandas as pd
import numpy as np
import time
import os
import positional_features
from IPython.display import clear_output

DATASET_OLD_PATH = '../data_old/dataset_bfnorm.csv'
DATASET_PATH = '../data/dataset.csv'
data = pd.read_csv(DATASET_OLD_PATH, index_col=0)

In [3]:
data

,fen,move,prob,eval_before,eval_after,game_phase,material,king_danger,center_control,file_control,pawn_structure
0,rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR ...,g7g6,0.675,0.18,0.46,0,0,0.25,15.00,0.00,2
1,rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR ...,c7c6,0.169,0.18,0.36,0,0,0.25,15.00,0.00,2
2,rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR ...,e7e6,0.039,0.18,0.30,0,0,0.25,15.00,0.00,2
3,rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR ...,g8f6,0.039,0.18,0.29,0,0,0.25,15.00,0.00,2
4,rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR ...,c7c5,0.039,0.18,0.15,0,0,0.25,15.00,0.00,2
...,...,...,...,...,...,...,...,...,...,...,...
105288,r1b2rk1/pq1p1p1p/6p1/2pp4/2P5/4PNP1/R2QBPKP/R7...,d2d5,1.000,4.61,4.38,1,1,1.12,-4.25,0.25,2
105289,r1b2rk1/p1qp1p1p/6p1/2pQ4/2P5/4PNP1/R3BPKP/R7 ...,d5a8,1.000,5.33,4.27,1,2,18.92,30.50,1.25,1
105290,Q4rk1/pbqp1p1p/6p1/2p5/2P5/4PNP1/R3BPKP/R7 w -...,a8a7,1.000,4.71,5.28,1,7,-60.24,6.75,1.25,1
105291,5rk1/Q1qp1p1p/6p1/2p5/2P5/4PbP1/R3BPKP/R7 w - ...,e2f3,1.000,6.30,6.38,1,5,-150.59,8.50,6.00,1


In [10]:
norm_king_danger = []
norm_center_control = []
norm_file_control = []

for index, row in data.iterrows():
    if index % 1000 == (len(data) - 1) % 1000:
        clear_output()
        print(f'...[{(index + 1) / len(data) * 100:.2f}%]')
        
    fen = row['fen']
    board = chess.Board(fen)
    king_danger = row['king_danger']
    center_control = row['center_control']
    file_control = row['file_control']
    
    norm_king_danger.append(np.around(np.tanh(king_danger / 150), 3))
    norm_center_control.append(np.around(np.tanh(center_control / 25), 3))
    norm_file_control.append(np.around(np.tanh(file_control / 1.5), 3))
    
data['norm_king_danger'] = norm_king_danger
data['norm_center_control'] = norm_center_control
data['norm_file_control'] = norm_file_control

data    

...[100.00%]


,fen,move,prob,eval_before,eval_after,game_phase,material,king_danger,center_control,file_control,pawn_structure,norm_king_danger,norm_center_control,norm_file_control
0,rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR ...,g7g6,0.675,0.18,0.46,0,0,0.25,15.00,0.00,2,0.002,0.537,0.000
1,rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR ...,c7c6,0.169,0.18,0.36,0,0,0.25,15.00,0.00,2,0.002,0.537,0.000
2,rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR ...,e7e6,0.039,0.18,0.30,0,0,0.25,15.00,0.00,2,0.002,0.537,0.000
3,rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR ...,g8f6,0.039,0.18,0.29,0,0,0.25,15.00,0.00,2,0.002,0.537,0.000
4,rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR ...,c7c5,0.039,0.18,0.15,0,0,0.25,15.00,0.00,2,0.002,0.537,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105288,r1b2rk1/pq1p1p1p/6p1/2pp4/2P5/4PNP1/R2QBPKP/R7...,d2d5,1.000,4.61,4.38,1,1,1.12,-4.25,0.25,2,0.007,-0.168,0.165
105289,r1b2rk1/p1qp1p1p/6p1/2pQ4/2P5/4PNP1/R3BPKP/R7 ...,d5a8,1.000,5.33,4.27,1,2,18.92,30.50,1.25,1,0.125,0.840,0.682
105290,Q4rk1/pbqp1p1p/6p1/2p5/2P5/4PNP1/R3BPKP/R7 w -...,a8a7,1.000,4.71,5.28,1,7,-60.24,6.75,1.25,1,-0.381,0.264,0.682
105291,5rk1/Q1qp1p1p/6p1/2p5/2P5/4PbP1/R3BPKP/R7 w - ...,e2f3,1.000,6.30,6.38,1,5,-150.59,8.50,6.00,1,-0.763,0.327,0.999


In [11]:
data.to_csv(DATASET_PATH)